# Change battery form factor
In this example we will redesign a battery form factor. Thereafter compare our virtual cells to the baseline

In [40]:
from breathe_design import api_interface as api
from breathe_design import enable_notebook_plotly
from breathe_design import define_cuboid_format, define_cylinder_format
from breathe_design import Cycler

enable_notebook_plotly()

We will begin with load the format of the Molicel P45B

In [41]:
base_format = api.get_battery_format("Molicel P45B")


🔐 No active session. Please sign in to continue.

🚀 Open this URL in a browser and verify the code shown is DWNR-XCNQ
🔗 https://breathe-platform.uk.auth0.com/activate?user_code=DWNR-XCNQ

⏳ Waiting for user authorization...

✅ Authentication successful!


We will now apply chnages to the format we do this with `get_updated_format` method. We will generate 10% shorter cell compared to baseline

In [42]:
smaller_format = api.get_updated_format(
    base_battery="Molicel P45B",
    name="smaller_format",
    height_mm=0.9 * base_format["height_mm"],
)

We will also create a cell with 10% smaller diameter

In [43]:
thinner_format = api.get_updated_format(
    base_battery="Molicel P45B",
    name="thinner_format",
    diameter_mm=0.9 * base_format["diameter_mm"],
)

In [44]:
formats = [smaller_format, thinner_format]

In [45]:
formats

[{'shape': 'cylinder',
  'material': 'steel',
  'housingThickness_mm': 0.18,
  'diameter_mm': 21.21,
  'height_mm': 63.0,
  'innerDiameter_mm': 2.3,
  'name': 'smaller_format'},
 {'shape': 'cylinder',
  'material': 'steel',
  'housingThickness_mm': 0.18,
  'diameter_mm': 19.089000000000002,
  'height_mm': 70,
  'innerDiameter_mm': 2.3,
  'name': 'thinner_format'}]

Now we can calculate all the equilibrium KPIs for each format using the `get_eqm_kpis` function. Instead of passing multiple designs as in previous examples, we now pass multiple formats.

In [46]:
results = api.get_eqm_kpis("Molicel P45B", designs=[], formats=formats)

In [47]:
results.get_kpis()

Baseline  smaller_format  thinner_format
KPI                                                                           
Capacity [Ah]                         4.500000        3.961879        3.478236
Nominal Voltage [V]                   3.656630        3.656630        3.656630
Energy [Wh]                          16.454837       14.487126       12.718624
Gravimetric Energy Density [Wh/kg]  241.196608      237.492427      228.904608
Volumetric Energy Density [Wh/l]    665.310409      650.834353      634.871516
Minimum Anode Voltage [mV]           80.490746       80.490746       80.490746
Weight [g]                           68.221675       61.000370       55.562987
Volume [l]                            0.024733        0.022259        0.020033
Heat Capacity [kJ/K]                  0.052220        0.046457        0.042006

These can be plotted using the `compare_designs` method in two different ways: one for a relative change in the KPIs and one for a delta change in the KPIs.

In [48]:
results.compare_designs("relative")

In [49]:
results.compare_designs("delta")

In [50]:
results.plot_radar()

Now lets create some brand new formats from scratch using the same volume for a cuboid pouch and a cylindrical 21700 cell.

In [51]:
f_cuboid = define_cuboid_format(
    name="my_cuboid",
    material="pouch",
    housingThickness_mm=1.0,
    length_mm=60.0,
    width_mm=40.0,
    thickness_mm=10.1,
    assemblyType="wound",
)

In [52]:
f_cylinder = define_cylinder_format(
    name="my_cylinder",
    material="aluminum",
    housingThickness_mm=1.0,
    diameter_mm=21.0,
    height_mm=70.0,
    innerDiameter_mm=5.0,
)

In [53]:
results = api.get_eqm_kpis("Molicel P45B", designs=[], formats=[f_cuboid, f_cylinder])

In [54]:
results.get_kpis()

Baseline   my_cuboid  my_cylinder
KPI                                                                    
Capacity [Ah]                         4.500000    3.594903     3.499849
Nominal Voltage [V]                   3.656630    3.656630     3.656630
Energy [Wh]                          16.454837   13.145230    12.797653
Gravimetric Energy Density [Wh/kg]  241.196608  203.540678   202.104862
Volumetric Energy Density [Wh/l]    665.310409  531.494704   527.841841
Minimum Anode Voltage [mV]           80.490746   80.490746    80.490746
Weight [g]                           68.221675   64.582817    63.321845
Volume [l]                            0.024733    0.024733     0.024245
Heat Capacity [kJ/K]                  0.052220    0.055066     0.054005

In [55]:
results.plot_radar()

We can also run dynamic simulations when applying different formats to cells.

In [56]:
cycler = Cycler(selected_unit="C", cell_capacity=4.5)

In [57]:
output = api.run_sim(
    "Molicel P45B", cycler.cc_dch(-1.0, 2.5), formats=[f_cuboid, f_cylinder]
)

In [58]:
output.plot_voltage_response()